In [4]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss',max_d=3):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cls_auc_res = [0,0,0,0,0,0]
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x

            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': max_d,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 60
            else:
                s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i], lr, feature_fraction, max_d)
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        cls_auc_res[i] = val_auc_l
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred, cls_auc_res

print('done')

file path ../features/fasttext_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_5_feat.pkl
(159571, 6) (15

In [5]:
lgb_res,tmp_auc_res = simple_ens('lgb',10,666,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# add 7 base fasttext NN models
# all loss avg 0.0319116484218 0.0358161833583 all auc avg 0.994546891137 0.991249510282 PUB 9866

# rm tfidf test
# all loss avg 0.0319555637279 0.0357756335619 all auc avg 0.994512718368 0.991251471241

# change to stratified
# all loss avg 0.0316412744167 0.0357232681944 all auc avg 0.994584962017 0.991293307537 pub 9866

# change to base model 5 fold, add word batch, tilli, lgb feat
# feat dim 217
# all loss avg 0.031983129767200906 0.035387924581 all auc avg 0.9945427088311004 0.99188514127 PUB 9870

# add more base models
# feat dim 247, all loss avg 0.031885221777487156 0.0353958  all auc avg 0.9945986685511962 0.9919487331094685

# change early stopping to 30, and test later part
# all loss avg 0.03208696729295824 0.035406264613808025 all auc avg 0.9945261773637045 0.991946174432887

# add fasttext lstm v1
# all loss avg 0.031977558955160815 0.03537909655655411 all auc avg 0.9945299073418443 0.9919307178575658

# add muse base model, feat dim 295, lower loss, but lower auc
# all loss avg 0.03196610276261484 0.03530962013098028 all auc avg 0.9945635742334386 0.9918682952070021

0 fold: ls 0.0701452724888359 0.07799274013322345 auc 0.9905986050450313 0.9871054558039831
1 fold: ls 0.07007731504136679 0.07154207938668194 auc 0.9905919721982126 0.9897577513585603
2 fold: ls 0.07061349029250148 0.07602982624979221 auc 0.9904227446612598 0.9883437433748105
3 fold: ls 0.06931647037392062 0.07322326840017794 auc 0.9908508658422951 0.9890206678734704
4 fold: ls 0.06812558906497987 0.07835384154450283 auc 0.9911960596148819 0.987742930639736
5 fold: ls 0.06965230897618752 0.07646615787767941 auc 0.9907760243808521 0.9876632403782849
6 fold: ls 0.0694829914573264 0.07788357054890263 auc 0.990786925584191 0.9875776118777836
7 fold: ls 0.07196767900922992 0.07091523519955178 auc 0.9898936997493964 0.9900605574634038
8 fold: ls 0.06851295097164081 0.07424784193449253 auc 0.9910957013883829 0.9893987832475472
9 fold: ls 0.06692787708051355 0.07751200288404718 auc 0.9915481516925352 0.9877562703424284
toxic 0.05 0.6 3
this class avg train 0.06948219447565027 avg val 0.075416

In [3]:
# find best params for each column, early stopping = 30

best_pred = np.zeros((153164,6))
val_auc_res = [0,0,0,0,0,0]
for lr in [0.095,0.075,0.05]:
    for max_d in [3,4]:
        for s_rate in [0.4,0.5,0.6]:
            print('learning rate',lr,'max depth',max_d,'feature fraction',s_rate)
            lgb_res,tmp_auc_res = simple_ens('lgb',k=10,rnd=666,lr=lr,
                                 feature_fraction=s_rate,bagging_fraction=0.9,
                                 bag_frec=3,met='binary_logloss',max_d=max_d)
            # check for each cls
            for i in range(6):
                # find better params for this class
                if tmp_auc_res[i] > val_auc_res[i]:
                    val_auc_res[i] = tmp_auc_res[i]
                    best_pred[:,i] = lgb_res[:,i]
                    print('FIND BETTER PARAMS',lr,max_d,s_rate,list_classes[i])
            print('TEST PARAM DONE ------------------------------------------')

print(val_auc_res)
print(np.mean(val_auc_res))
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = best_pred
sample_submission.to_csv("../results/lgb_grid_search_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
                
            
# best auc params
# toxic 0.05,4,0.5
# severe toxic 0.075 3 0.5
# obs 0.075 3 0.6
# threat 0.095 3 0.5
# insult 0.075 0.5 4
# hate 0.05 0.5 3

# TEST PARAM DONE ------------------------------------------
# [0.9884873039634368, 0.9920148292218363, 0.9954573112511824, 
#  0.994769628570376, 0.9898761527824232, 0.9912880359235366]
# 0.9919822102854652 PUB 9870

learning rate 0.095 max depth 3 feature fraction 0.4
0 fold: ls 0.07055900792121056 0.07845851850483938 auc 0.9904317585515335 0.9869234386296798
1 fold: ls 0.07112099324841784 0.07132731780698526 auc 0.99019070161143 0.9898649322033591
2 fold: ls 0.06935924063822202 0.07556019794440967 auc 0.9908043936706946 0.9885162021559386
3 fold: ls 0.06497260357247332 0.07280091655739042 auc 0.992192585088142 0.9892247916632692
4 fold: ls 0.06894506079113848 0.07792721171645994 auc 0.9909000263031499 0.9878534906782339
5 fold: ls 0.06577492446228168 0.07645398182283704 auc 0.9919903108446425 0.9876221260056249
6 fold: ls 0.06797329110174438 0.07772442055333846 auc 0.9912493848473811 0.9876035860073693
7 fold: ls 0.07310551182282535 0.07122791366621857 auc 0.9894850349134301 0.9899879336591975
8 fold: ls 0.0704343227293608 0.07448200464055142 auc 0.9904142517680505 0.9892556209668459
9 fold: ls 0.06958413131063694 0.0776744481375048 auc 0.9906593620485566 0.9876591212710091
toxic 0.095 0.4 3
this

toxic 0.095 0.5 3
this class avg train 0.06938118599908744 avg val 0.07559496416987946
this class auc train 0.990786140903821 auc val 0.9883672816369625
0 fold: ls 0.017906577710877336 0.021184588850506145 auc 0.9942284572075157 0.9911464267628813
1 fold: ls 0.017322633815901163 0.02052464275821394 auc 0.9947602536605363 0.9917010855804532
2 fold: ls 0.017502525843847296 0.021137535246129414 auc 0.9946054788366333 0.9912144733510572
3 fold: ls 0.018127065755140153 0.019999904463978653 auc 0.9940710283026972 0.9924674009368275
4 fold: ls 0.017554294298551983 0.021112351147607965 auc 0.9945542352245011 0.9910142897835168
5 fold: ls 0.01778366661478703 0.019919998316783 auc 0.9943610227338315 0.9922755129420888
6 fold: ls 0.018160210122930027 0.01903681796179376 auc 0.9940604463035052 0.9930784565017718
7 fold: ls 0.017106348494256657 0.020023243398108247 auc 0.9949557166952303 0.9922459602432275
8 fold: ls 0.016380160382229014 0.020935282161855716 auc 0.9955436682164217 0.991344188829739

0 fold: ls 0.03494977666811921 0.03725287503188428 auc 0.9963043753227504 0.9956300015230432
1 fold: ls 0.03582070852237579 0.03698556199148193 auc 0.996124820336235 0.9955101157082131
2 fold: ls 0.03195444835935681 0.03878083435607207 auc 0.9969636253615817 0.9954401220394625
3 fold: ls 0.035835934813539486 0.033893640961965156 auc 0.9961095720984094 0.9964988441334289
4 fold: ls 0.034164538032889824 0.04198606043703194 auc 0.9964842602547134 0.9945381388981992
5 fold: ls 0.03389509653876179 0.03753737316766365 auc 0.9965324424577965 0.9956091166234913
6 fold: ls 0.03172194789132538 0.03732420149605098 auc 0.9970218421797358 0.995228448100338
7 fold: ls 0.033759061065030196 0.03842424606720479 auc 0.9965684942635612 0.9954971322605805
8 fold: ls 0.034652884829612055 0.03864862489473035 auc 0.9963780492979039 0.9951816965865913
9 fold: ls 0.03507308709139142 0.03881393310410665 auc 0.9962790583571295 0.9953099793265576
obscene 0.095 0.6 3
this class avg train 0.0341827483812402 avg val

1 fold: ls 0.003988048583730065 0.006114999226034538 auc 0.999314627288643 0.9971048082966687
2 fold: ls 0.004238654843538619 0.007390458196229382 auc 0.9991249448380337 0.9949481458202388
3 fold: ls 0.004195360454438888 0.00669774687316152 auc 0.9992271937620676 0.9912588786221636
4 fold: ls 0.004039453325756143 0.0066911300332559605 auc 0.9992843652792178 0.9963765269134872
5 fold: ls 0.004025442758317083 0.007049136747562741 auc 0.9992613748580457 0.9959155718985062
6 fold: ls 0.004287852739327959 0.006308494747856267 auc 0.9991120548742616 0.9945863976365578
7 fold: ls 0.004140903865941209 0.007490696191338755 auc 0.9991697623743852 0.9960203344019108
8 fold: ls 0.00417165748513597 0.0071405796059674795 auc 0.9991831936788489 0.9938988101208603
9 fold: ls 0.004288430522231775 0.007670597807387139 auc 0.9989170230947786 0.9812417165180153
threat 0.095 0.4 4
this class avg train 0.004124727405696272 avg val 0.006912003001893856
this class auc train 0.9991952785286242 auc val 0.993821

2 fold: ls 0.04967580845953204 0.055093581536661836 auc 0.9919432575381819 0.9891918661263714
3 fold: ls 0.04579873204731029 0.05208357617904119 auc 0.9932878818836879 0.9903970734384254
4 fold: ls 0.04699117132954316 0.05523596326562538 auc 0.9928615097516653 0.9891971771175049
5 fold: ls 0.04347723998202296 0.05249802343633111 auc 0.994073364809558 0.9904509029067765
6 fold: ls 0.04655846467277939 0.056300705171345854 auc 0.9929967830726962 0.9891309185545059
7 fold: ls 0.04601123886833809 0.05564600522357967 auc 0.9932005494015907 0.9894426228574409
8 fold: ls 0.04838821809640416 0.05257368713625432 auc 0.9923729556602027 0.9903932006048247
9 fold: ls 0.047665586149476934 0.05468082850132548 auc 0.9926200148817788 0.9892286004619031
insult 0.095 0.5 4
this class avg train 0.04694683043433662 avg val 0.05380147843094686
this class auc train 0.9928738470768923 auc val 0.9898644947687915
0 fold: ls 0.012569690637461956 0.01715887328501066 auc 0.9972245879066522 0.9927688002450007
1 fol

...

5 fold: ls 0.015225525120347432 0.01648194753164859 auc 0.995075861122598 0.9935363397429529
6 fold: ls 0.014929549026047869 0.01660500624739647 auc 0.9953149352136844 0.9904368993424381
7 fold: ls 0.01530657321175264 0.018733163090437883 auc 0.9949995987732736 0.9881711106293809
8 fold: ls 0.014983140549707242 0.016769288799966767 auc 0.9953577845620728 0.9929379832357829
9 fold: ls 0.015187936719845115 0.016724422181596477 auc 0.9951386870028919 0.9919313172917118
identity_hate 0.05 0.4 3
this class avg train 0.01512720981200696 avg val 0.017522501569163913
this class auc train 0.9951690035095228 auc val 0.9911234286356475
all loss avg 0.03200428095230993 0.03531787410896697
all auc avg 0.994509481713925 0.9919197287224049
TEST PARAM DONE ------------------------------------------
learning rate 0.05 max depth 3 feature fraction 0.5
0 fold: ls 0.06849146334705691 0.07813722148395294 auc 0.991118156224512 0.9870697137555695
1 fold: ls 0.06922922958231303 0.07134048396590319 auc 0.99087

3 fold: ls 0.0665333324130331 0.07302716424674889 auc 0.991738939658409 0.9891501365355311
4 fold: ls 0.06816725795753126 0.07819660633695175 auc 0.9911647930312595 0.9877842263326724
5 fold: ls 0.06980775600733378 0.07671841633335064 auc 0.9906886905494074 0.9876456523114799
6 fold: ls 0.06925027616904056 0.07798349729060562 auc 0.9908400292622995 0.9875269781906159
7 fold: ls 0.07158440763155928 0.07118364109136627 auc 0.9900607409622522 0.9899797283479856
8 fold: ls 0.06991973579023937 0.07433905542779407 auc 0.9905886098142526 0.9893620633465439
9 fold: ls 0.07021564971809863 0.07798751259156836 auc 0.9904586806341834 0.9874957403781506
toxic 0.05 0.6 3
this class avg train 0.06970609377297203 avg val 0.07553991979772219
this class auc train 0.9906879539041373 auc val 0.9884020376502374
0 fold: ls 0.017822169135592737 0.021269788641738945 auc 0.9943041266963633 0.9910617641473604
1 fold: ls 0.017911115735389685 0.020538599537768817 auc 0.9942413109973594 0.9916615236105837
2 fold: 

4 fold: ls 0.01654437499871801 0.021213404341519547 auc 0.9955255332631818 0.9910530605139891
5 fold: ls 0.015864456927957824 0.01974344935655945 auc 0.9960539137040779 0.9924789460651416
6 fold: ls 0.016746579200130687 0.019140144088081484 auc 0.9953838839545138 0.992980117632398
7 fold: ls 0.015104627983725773 0.020148477082327556 auc 0.996656442197966 0.9921002435380016
8 fold: ls 0.01617572175306264 0.02117403061046551 auc 0.995783871133765 0.9910650975445938
9 fold: ls 0.01676547005493644 0.019020745927220875 auc 0.9953819541810658 0.9931628607135419
severe_toxic 0.05 0.4 4
this class avg train 0.016167035852938157 avg val 0.0203795824635778
this class auc train 0.9958200077698258 auc val 0.9918986752770549
0 fold: ls 0.03285726014331015 0.03711731370247829 auc 0.9967867563913145 0.9956462890798587
1 fold: ls 0.03357447895290844 0.037057298069595915 auc 0.9966411707313872 0.9955063570412557
2 fold: ls 0.028880456483688803 0.038674211891549376 auc 0.9976191265835459 0.9954366763667

5 fold: ls 0.03305838139808371 0.03757879506952878 auc 0.9967376357356217 0.9955494438371011
6 fold: ls 0.034369530778807174 0.03750568713800894 auc 0.9964979585002047 0.9954744221450251
7 fold: ls 0.03372839105790439 0.03837946160459001 auc 0.9966109315576597 0.9955363659429709
8 fold: ls 0.032308930427461635 0.03886685264388981 auc 0.9969061704702608 0.9951658778164458
9 fold: ls 0.03324655142934531 0.038879050676179604 auc 0.9967062536154742 0.9952615259459229
obscene 0.05 0.5 4
this class avg train 0.033264506133872315 avg val 0.038073530144331916
this class auc train 0.9967063786292512 auc val 0.9954255719044036
0 fold: ls 0.003946354090892078 0.006654386465549525 auc 0.9993233898593585 0.9968429184998953
1 fold: ls 0.0038432936144718235 0.006218776454716906 auc 0.9994001821380316 0.9969817200921851
2 fold: ls 0.004198176740596978 0.00756835366228326 auc 0.9991401311283382 0.9944387701655142
3 fold: ls 0.0036097312203227415 0.00653086733816286 auc 0.9995228046137414 0.991529951599

6 fold: ls 0.0042210360921609204 0.006459649166626927 auc 0.9991572642259627 0.9941974668426677
7 fold: ls 0.00408061850244914 0.0076289121528927105 auc 0.999215385894747 0.9957139040794519
8 fold: ls 0.004255438775326833 0.007217201995839656 auc 0.999103752470496 0.9938653752793482
9 fold: ls 0.004221652970011515 0.007595792457694835 auc 0.9991527540439911 0.9801310110829813
threat 0.05 0.6 4
this class avg train 0.004084060403800594 avg val 0.006986319144707013
this class auc train 0.9992430000654983 auc val 0.9935584649192869
0 fold: ls 0.04882326381997676 0.05017206073345681 auc 0.9922173930942987 0.9913672122041566
1 fold: ls 0.048753144858656165 0.05408061509492768 auc 0.9922485611240078 0.9897597114261718
2 fold: ls 0.04952517165495439 0.05508197843804442 auc 0.9919988939952588 0.9890663010416632
3 fold: ls 0.04729200169449892 0.052139512120979184 auc 0.9927737736804646 0.9903744867809497
4 fold: ls 0.048734877664425996 0.05532769868744674 auc 0.9922610736555902 0.98903337122564

In [ ]:
def special_ens(model_name,k=3,rnd=233):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    params_list = [
        [0.05,4,0.5],
        [0.075,3,0.5],
        [0.075,3,0.6],
        [0.095,3,0.5],
        [0.075,4,0.5],
        [0.05,3,0.5],
    ]
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        
        # special params
        params = {
                'application': 'binary',
                #'num_leaves': 8,
                #'lambda_l1': 1,
                'lambda_l2': 1.0,
                'max_depth': params_list[i][1],
                'metric': 'binary_logloss', # or auc
                'data_random_seed': 2,
                'learning_rate':params_list[i][0],
                'feature_fraction': params_list[i][2],

                }
        print(params)
            
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i])
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

lgb_res = special_ens('lgb',10,666)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified_special.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# best params changed when base models changed
# all loss avg 0.03111512578966158 0.03534320053008906 all auc avg 0.9948524214184179 0.9918450388634261